In [7]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action = 'ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [20]:
data = pd.read_csv('data/201901-202003.csv')
data = data.fillna('')

In [5]:
df = data.copy()
df = df[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
df = df.loc[df['REG_YYMM']==202003]
df = df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.loc[submission['REG_YYMM']==202004]
submission = submission[['CARD_SIDO_NM', 'STD_CLSS_NM']]
submission = submission.merge(df, left_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission = submission.fillna(0)
AMT = list(np.expm1(np.log1p(submission.AMT.values+1)))*2

submission = pd.read_csv('submission.csv', index_col=0)
submission['AMT'] = AMT
submission.to_csv('dacon_submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.605901e+07
1,202004,강원,골프장 운영업,2.915798e+09
2,202004,강원,과실 및 채소 소매업,9.948169e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,1.331730e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.000000e+00


In [21]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [22]:
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [23]:
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [35]:
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3


In [37]:
sido = pd.DataFrame(df.groupby('CARD_SIDO_NM')['AMT'].sum())
clss = pd.DataFrame(df.groupby('STD_CLSS_NM')['AMT'].sum())
age = pd.DataFrame(df.groupby('AGE')['AMT'].sum())
sex = pd.DataFrame(df.groupby('SEX_CTGO_CD')['AMT'].sum())

In [38]:
pd.DataFrame(df.groupby('FLC')['AMT'].sum())

,AMT
FLC,
1,7371507278072
2,12619055902225
3,11538603588734
4,17864707502682
5,10902152777293


In [39]:
# 라이브러리 불러오기
from sklearn.preprocessing import OneHotEncoder

# numpy를 사용해 배열로 변경
arr = np.array(df['FLC'])

# OneHotEncoder를 넣기 위해 배열의 shape변경
arr = arr.reshape(-1,1)

# OneHotEncoder 생성
encoder = OneHotEncoder()

# OneHotEncoder 학습시키기
encoder.fit(arr)

# OneHotEncoder 적용시키기
enc = encoder.transform(arr).toarray()

In [40]:
enc

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [47]:
exo2 = pd.DataFrame(enc, columns=['1', '2', '3','4','5'])
df = pd.concat([exo2, df], axis=1)

In [48]:
df['abs'] = abs(df['CSTMR_CNT']-df['CNT'])
df.tail()

,1,2,3,4,5,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT,abs
1057389,0.0,0.0,0.0,0.0,1.0,충북,휴양콘도 운영업,충북,70s,2,5,2019,3,3,148000,4,1
1057390,0.0,0.0,0.0,0.0,1.0,충북,휴양콘도 운영업,충북,70s,2,5,2019,5,5,329800,7,2
1057391,0.0,0.0,0.0,0.0,1.0,충북,휴양콘도 운영업,충북,70s,2,5,2019,10,7,557800,7,0
1057392,0.0,0.0,0.0,0.0,1.0,충북,휴양콘도 운영업,충북,70s,2,5,2019,12,3,247800,3,0
1057393,0.0,0.0,0.0,0.0,1.0,충북,휴양콘도 운영업,충북,70s,2,5,2020,1,3,230400,3,0


In [28]:
y_data = df['AMT']


TypeError: string indices must be integers

In [49]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [50]:
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [51]:
k = int(len(x)*0.9)

In [52]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [53]:
import lightgbm as lgb

In [54]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [55]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [56]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.798722
[200]	valid_0's rmse: 0.730291
[300]	valid_0's rmse: 0.707395
[400]	valid_0's rmse: 0.69189
[500]	valid_0's rmse: 0.681943
[600]	valid_0's rmse: 0.672866
[700]	valid_0's rmse: 0.665098
[800]	valid_0's rmse: 0.658457
[900]	valid_0's rmse: 0.652299
[1000]	valid_0's rmse: 0.647716
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.647716


In [57]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

ValueError: Shape of passed values is (1658860, 8), indices imply (1658860, 14)

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()